# Amanogawa Notebook 03: Astronomical Validity (Run Manifest Driven)

This notebook performs lightweight consistency checks using outputs generated by:

```bash
amanogawa-run --image-dir data/raw --out outputs/run --recursive
```

It does not implement scientific core logic; it only reads exported artifacts.


In [ ]:
from __future__ import annotations

import json
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

ROOT_CANDIDATES = [Path("..").resolve(), Path(".").resolve()]
ROOT = next((p for p in ROOT_CANDIDATES if (p / "src" / "amanogawa").exists()), ROOT_CANDIDATES[0])
RUN_DIR = ROOT / "outputs" / "run"
MANIFEST_PATH = RUN_DIR / "run_manifest.json"

if not MANIFEST_PATH.exists():
    raise FileNotFoundError(
        "run_manifest.json not found. Run `amanogawa-run --image-dir data/raw --out outputs/run --recursive` first."
    )

manifest = json.loads(MANIFEST_PATH.read_text(encoding="utf-8"))
print(f"Run status: {manifest['status']}")
print(f"Images: {manifest['num_images']} (ok={manifest['num_images_ok']}, error={manifest['num_images_error']})")


In [ ]:
rows = []
for image_item in manifest.get("images", []):
    steps = image_item.get("steps", {})
    rows.append(
        {
            "image": image_item.get("image"),
            "detected_stars": image_item.get("detected_stars"),
            "detect": steps.get("detect", {}).get("status"),
            "stats": steps.get("stats", {}).get("status"),
            "band": steps.get("band", {}).get("status"),
            "dark": steps.get("dark", {}).get("status"),
        }
    )

status_df = pd.DataFrame(rows)
status_df


In [ ]:
ok_items = [item for item in manifest.get("images", []) if item.get("status") == "ok"]
if not ok_items:
    raise RuntimeError("No successful image entry found in run_manifest.json")

target = ok_items[0]
steps = target["steps"]

detect_path = Path(steps["detect"]["summary_json"])
stats_path = Path(steps["stats"]["summary_json"])
band_path = Path(steps["band"]["summary_json"])
dark_path = Path(steps["dark"]["summary_json"])

detection = json.loads(detect_path.read_text(encoding="utf-8"))
stats = json.loads(stats_path.read_text(encoding="utf-8"))
band = json.loads(band_path.read_text(encoding="utf-8"))
dark = json.loads(dark_path.read_text(encoding="utf-8"))

metric_series = pd.Series(
    {
        "image": target["image"],
        "num_stars": detection.get("num_stars"),
        "nnd_mean": stats.get("nearest_neighbor", {}).get("mean"),
        "xi_mean": stats.get("two_point_correlation", {}).get("xi_mean"),
        "band_angle_deg": band.get("principal_axis", {}).get("angle_deg"),
        "band_empirical_fwhm_px": band.get("band_width_measurements", {}).get("empirical_fwhm_px"),
        "dark_area_fraction": dark.get("dark_area_fraction"),
        "dark_components": dark.get("num_dark_components"),
    }
)
metric_series


In [ ]:
checks = [
    {
        "metric": "detected stars > 0",
        "value": float(detection.get("num_stars", 0)),
        "criterion": "> 0",
        "passed": float(detection.get("num_stars", 0)) > 0,
    },
    {
        "metric": "stats status",
        "value": stats.get("status"),
        "criterion": "ok or no_detections",
        "passed": stats.get("status") in {"ok", "no_detections"},
    },
    {
        "metric": "band status",
        "value": band.get("status"),
        "criterion": "ok or no_detections",
        "passed": band.get("status") in {"ok", "no_detections"},
    },
    {
        "metric": "dark area fraction",
        "value": float(dark.get("dark_area_fraction", -1.0)),
        "criterion": "0 <= x <= 1",
        "passed": 0.0 <= float(dark.get("dark_area_fraction", -1.0)) <= 1.0,
    },
]

checks_df = pd.DataFrame(checks)
score = float(checks_df["passed"].mean()) if len(checks_df) else 0.0
print(f"Validity score: {score:.2f}")
checks_df


In [ ]:
fig, ax = plt.subplots(figsize=(8, 3.5))
bar_colors = ["#2ecc71" if p else "#e74c3c" for p in checks_df["passed"]]
ax.barh(checks_df["metric"], [1] * len(checks_df), color=bar_colors)
ax.set_xlim(0, 1)
ax.set_xlabel("Pass/Fail")
ax.set_title(f"Astronomical validity quick checks (score={score:.2f})")
for i, passed in enumerate(checks_df["passed"]):
    ax.text(0.5, i, "PASS" if passed else "FAIL", ha="center", va="center", color="white", fontweight="bold")
plt.tight_layout()
plt.show()
